In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.edge.service import Service
import requests
import time
import random
import datetime
import pandas as pd

In [2]:
# 配置文件部分
# 浏览器位置
broswer_path = 'msedgedriver.exe'
# cookie
cookie_path = '/html/body/div[7]/div[2]/div/div[1]/div/div[2]/div/button[2]'
# 目标url
url = "https://www.proquest.com/advanced?accountid=171768"
# 搜索框
queryfield_path = '#queryTermField'
queryfield_path_2 = '//*[@id="queryTermField_0"]'
# 查询文本
wenben = '(China OR Chinese) AND (("nuclear war*" OR "atomic war*" OR "nuclear missile*" OR "nuclear bomb*" OR "atomic bomb*" OR "h-bomb*" OR "hydrogen bomb*" OR "nuclear test" OR "nuclear weapon*" OR "nuclear proliferation" OR "nuclear program" OR "nuclear weapons" OR "China’s Nuclear Forces") OR (missile* OR "arms" OR "military power" OR "military equipment" OR WMD OR "weapons of mass destruction" OR "CBRN attack" OR "CBRN threats" OR "biological warfare" OR Biotechnology*) OR ("Internet of Things" OR IoT OR "Artificial Intelligence" OR AI OR "Augmented Reality" OR AR OR "Virtual Reality" OR VR OR "cyber eapability" OR "cyber attacks" OR "cyber threats" OR "cyber espionage" OR 5G OR telecommunication OR 6G OR 2G OR 3G OR 4G OR "digital economy" OR cryptocurrency OR blockchain OR bitcoin OR "distributed sensors" OR "edge computing" OR "autonomous systems" OR "quantum information science" OR QIS OR quantum OR "quantum information*" OR quantum* OR gis* OR "advanced microelectronics" OR "BeiDou Navigation Satellite System") OR ("precision medicine" OR "genomic technologies" OR "space technologies" OR "undersea technologies" OR Robotics OR "decentralized energy methods" OR Nanotechnology OR "new materials"))'
wenben_2 = '35024 OR 10482 OR 105983 OR 29263 OR 105397 OR 33677 OR 10337 OR 17099 OR 55090'
# 选定出版物类型中的报纸
newsPapers_xpath = '//*[@id="SourceType_Newspapers"]'
# 选定文档类型
article_xpath = '//*[@id="DocumentType_Article"]'
commentary_xpath = '//*[@id="DocumentType_Commentary"]'
editorial_xpath = '//*[@id="DocumentType_Editorial"]'
feature_xpath = '//*[@id="DocumentType_Feature"]'
frontPage_xpath = '//*[@id="DocumentType_Front_Page/Cover_Story"]'
news_xpath = '//*[@id="DocumentType_News"]'
report_xpath = '//*[@id="DocumentType_Report"]'
review_xpath = '//*[@id="DocumentType_Review"]'
# 选定语言
# 1.定位到语言选择框
js_code = 'document.getElementsByClassName("dySrchScroller")[2].scrollBy(0, document.body.scrollHeight)'
# 2. 定位英语
language_path = '#Language_ENG'
# 月份设置
month = ['JANUARY','FEBRUARY','MARCH','APRIL','MAY','JUNE','JULY','AUGUST','SEPTEMBER','OCTOBER','NOVEMBER','DECEMBER']
# 自定义日期路径
time_field_path = '//*[@id="select_multiDateRange"]'
# 日期输入框路径
day_path1 = '//*[@id="day2"]'
day_path2 = '//*[@id="day2_0"]'
month_path1 = '//*[@id="month2"]'
month_path2 = '//*[@id="month2_0"]'
year_path1 = '//*[@id="year2"]'
year_path2 = '//*[@id="year2_0"]'
# 搜索按钮路径
search_xpath = '//*[@id="searchToResultPage"]'
# 检索结果路径
result_xpath = '//*[@id="pqResultsCount"]'
# 我的pushdeer密钥
key = "PDU20716TZ4QHUXWH12stzo90lAJu97MIRRisiINQ"

In [3]:
# 定义浏览器
def set_broswer(driver_type = 'edge', path = broswer_path):
    if driver_type == 'edge':
        driver = webdriver.Edge(service=Service(path))
    elif driver_type == 'chrome':
        driver = webdriver.Chrome(path)
    else:
        raise ValueError (" use edge or chrome!")
    return driver

In [4]:
# 定义勾选复选框操作
def run_select_config():
    # Source type: Newspapers
    driver.find_element(By.XPATH, newsPapers_xpath).click()
    driver.implicitly_wait(20)

    driver.find_element(By.XPATH,article_xpath).click()
    driver.find_element(By.XPATH,commentary_xpath).click()
    driver.find_element(By.XPATH,editorial_xpath).click()
    driver.find_element(By.XPATH,feature_xpath).click()
    driver.find_element(By.XPATH,frontPage_xpath).click()
    driver.find_element(By.XPATH,news_xpath).click()
    driver.find_element(By.XPATH,report_xpath).click()
    driver.find_element(By.XPATH,review_xpath).click()
    driver.implicitly_wait(20)

    # 定位到语言选择框
    driver.execute_script(js_code)
    # 选择英语//
    driver.find_element(By.CSS_SELECTOR, language_path).click()
    driver.implicitly_wait(20)

In [5]:
# 定义保存函数
def save(result_list, begin_time, end_time, freq):
    """定义一个保存函数,时间格式："1990-01-01"

    Args:
        result_list (list): numresult
        begin_time (str): "1991-01"
        end_time (str): "1991-01"

    Returns:
        _type_: dataframe
    """
    date = pd.date_range(start=begin_time, end=end_time,freq=freq)
    temp = pd.DataFrame({"date":date,"index":result_list})
    time = "{}-{}".format(begin_time,end_time)
    df = pd.concat([history, temp])
    print(f"index({time}) have saved!")
    # temp.to_csv('index({}).csv'.format(time),index=False,header=True,encoding='utf8')
    return df

In [6]:
# 定义生成日期函数
def getDate(start, end, freq='M'):
    """_summary_

    Args:
        start_date (datetime): _description_
        end_date (datetime): _description_

    Returns:
        list: list(元组)：元组第一个元素是开始日期，第二个元素是结束日期
    """
    if freq == 'M':
        dates = []
        current_date = start
        while current_date <= end:
            if current_date.day == 1:
                last_day = current_date.replace(day=28) + datetime.timedelta(days=4)  # last day of month
                last_day = last_day - datetime.timedelta(days=last_day.day)  # adjust for shorter months
                dates.append((current_date.strftime("%Y-%#m-%#d"), last_day.strftime("%Y-%#m-%#d")))
            current_date += datetime.timedelta(days=1)
    elif freq == 'W':
        head = pd.date_range(start, end, freq='W-MON').strftime("%Y-%#m-%#d").map(str)
        tail = pd.date_range(start, end, freq='W-SUN').strftime("%Y-%#m-%#d").map(str)
        dates = list(zip(head, tail))
    elif freq == 'D':
        head = pd.date_range(start, end, freq='D').strftime("%Y-%#m-%#d").map(str)
        tail = pd.date_range(start+datetime.timedelta(days=1), end+datetime.timedelta(days=1), freq='D').strftime("%Y-%#m-%#d").map(str)
        dates = list(zip(head, tail))
    else:
        print("please use suitable freq!")
    return dates

In [7]:
# 定义发送信息函数
def sendMessage(pushkey, text):
    pushdeer = "https://api2.pushdeer.com/message/push"
    data = {
        "pushkey": pushkey,
        "text": text
    }
    resp = requests.get(pushdeer, data)
    if resp.status_code==200:
        print("send success!")
    else: 
        print(f"something wrong at {datetime.datetime.now().strftime('%Y-%m-%d  %H:%M:%S')}")

In [8]:
# 时间config
freq = 'W'
start_date = datetime.date(2012, 1, 2)
end_date = datetime.date(2023, 1, 8)  
date_list = getDate(start_date, end_date, freq)

In [9]:
# 主程序
# 初始化浏览器
flag = True
n = 0 
history = pd.DataFrame()
while flag:
    try:
        driver = set_broswer('edge')# 这个地方你换成"chrome",然后修改配置文件中的browser_path为你自己的路径就行
        driver.get(url)
        time.sleep(3)
        # 点击确认cookies
        cookie_path = '//*[@id="onetrust-accept-btn-handler"]'
        driver.find_element(By.XPATH, cookie_path).click()
        # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,cookie_path))).click()
        driver.implicitly_wait(20)
        # 刷新
        driver.refresh()
        driver.implicitly_wait(20)
        # 复选框勾选
        run_select_config()
        results_list = []
        # 选择自定义日期
        Select(driver.find_element(By.XPATH, time_field_path)).select_by_value('RANGE')
        driver.implicitly_wait(20)
        # 输入文本
        driver.find_element(By.CSS_SELECTOR, queryfield_path).send_keys(wenben)
        # 输入文本
        driver.find_element(By.XPATH, queryfield_path_2).send_keys(wenben_2)
        driver.implicitly_wait(20)
        # 主循环
        for (i,j) in date_list:
            # # 输入文本
            # driver.find_element(By.CSS_SELECTOR, queryfield_path).send_keys(wenben)
            # # 输入文本
            # driver.find_element(By.XPATH, queryfield_path_2).send_keys(wenben_2)
            # driver.implicitly_wait(20)
            
            # 年月日解包
            st_y, st_m, st_d = i.split('-')[0], i.split('-')[1], i.split('-')[2]
            ed_y, ed_m, ed_d = j.split('-')[0], j.split('-')[1], j.split('-')[2]
            
            # 输入开始日期
            driver.implicitly_wait(20)
            Select(driver.find_element(By.XPATH, day_path1)).select_by_value(st_d)
            time.sleep(random.random()+1.5)
            Select(driver.find_element(By.XPATH, month_path1)).select_by_value(month[int(st_m)-1])
            start_year = driver.find_element(By.XPATH, year_path1)
            start_year.clear()
            start_year.send_keys(st_y)
            
            # 输入结束时间
            driver.implicitly_wait(20)
            Select(driver.find_element(By.XPATH, day_path2)).select_by_value(ed_d)
            Select(driver.find_element(By.XPATH, month_path2)).select_by_value(month[int(ed_m)-1])
            end_year = driver.find_element(By.XPATH, year_path2)
            end_year.clear()
            end_year.send_keys(ed_y)

            if driver.find_element(By.XPATH, newsPapers_xpath).is_selected():
                pass
            else:
                run_select_config()
            
            # Click Search
            driver.find_element(By.XPATH, search_xpath).click()
            driver.implicitly_wait(20)
            
            # 返回搜索结果数量
            result = driver.find_element(By.XPATH, result_xpath).text
            print(i,' -->', j, ' : ',result)
            stop_date = j # 跟新stop_date 记录断点
            results_list.append(result)
            driver.implicitly_wait(20)

            # 找到高级搜索链接并点击跳转
            link_href = driver.find_element(By.XPATH,"//a[contains(text(),'高级检索')]").get_attribute('href')
            driver.get(link_href)
            driver.implicitly_wait(20)
    except Exception as e:
        # 错误计数
        n += 1
        print(f"第{n}次中断")
        # 发送信息给我
        sendMessage(key, f"第{n}次中断"+'-->'+str(e.args)+f'--> stop date:{j}')
        # 先保存
        history = save(results_list, start_date, stop_date, freq)
        # 重置开始时间
        start_date = datetime.date(int(st_y), int(st_m), int(st_d))
        date_list = getDate(start_date, end_date, freq) 
    else:
        history = save(results_list, start_date, end_date, freq)
        history.to_csv(f'index-{end_date}.csv', encoding='utf8', index=False, header=True)
        sendMessage(key, "Mission Completed!")
        flag = False


2012-1-2  --> 2012-1-8  :  72 个检索结果
2012-1-9  --> 2012-1-15  :  90 个检索结果
2012-1-16  --> 2012-1-22  :  76 个检索结果
2012-1-23  --> 2012-1-29  :  71 个检索结果
2012-1-30  --> 2012-2-5  :  69 个检索结果
2012-2-6  --> 2012-2-12  :  103 个检索结果
2012-2-13  --> 2012-2-19  :  100 个检索结果
2012-2-20  --> 2012-2-26  :  81 个检索结果
2012-2-27  --> 2012-3-4  :  99 个检索结果
2012-3-5  --> 2012-3-11  :  73 个检索结果
2012-3-12  --> 2012-3-18  :  82 个检索结果
2012-3-19  --> 2012-3-25  :  80 个检索结果
2012-3-26  --> 2012-4-1  :  93 个检索结果
2012-4-2  --> 2012-4-8  :  68 个检索结果
2012-4-9  --> 2012-4-15  :  101 个检索结果
2012-4-16  --> 2012-4-22  :  98 个检索结果
2012-4-23  --> 2012-4-29  :  64 个检索结果
2012-4-30  --> 2012-5-6  :  79 个检索结果
2012-5-7  --> 2012-5-13  :  58 个检索结果
2012-5-14  --> 2012-5-20  :  71 个检索结果
2012-5-21  --> 2012-5-27  :  76 个检索结果
2012-5-28  --> 2012-6-3  :  64 个检索结果
2012-6-4  --> 2012-6-10  :  59 个检索结果
2012-6-11  --> 2012-6-17  :  90 个检索结果
2012-6-18  --> 2012-6-24  :  65 个检索结果
2012-6-25  --> 2012-7-1  :  55 个检索结果
2012-7-2  --> 2012-7-8  :

In [10]:
history

,date,index
0,2012-01-08,72 个检索结果
1,2012-01-15,90 个检索结果
2,2012-01-22,76 个检索结果
3,2012-01-29,71 个检索结果
4,2012-02-05,69 个检索结果
...,...,...
570,2022-12-11,105 个检索结果
571,2022-12-18,132 个检索结果
572,2022-12-25,123 个检索结果
573,2023-01-01,98 个检索结果


In [11]:
history.to_csv('week_index_2012-0102-2023-0108.csv', header=True, index=False, encoding='utf-8')
    
    